In [1]:
pip install praw pandas matplotlib wordcloud nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw

reddit = praw.Reddit(
    client_id="3VypVlHsKWYOd43a4XZ1Qw",            # your client ID
    client_secret="RnwapkXWjkuee9Rv8B5rEoaOscL4wg", # your secret
    user_agent="NLP_SriLankaResearchApp by u/Cute_Firefighter1926"
)


In [2]:
print(reddit.user.me())   # should print "None" (since it's read-only)
for post in reddit.subreddit("srilanka").hot(limit=5):
    print(post.title)

None
Why on earth did Daily Mirror take this post down?
Double check before you buy from Abans 11.11 sale
Guess where this video is from.
Here we go. People opposing the implementation of Sex Education. How can we ever improve as a country with this mindset? What do you guys think?
One Shot One is a cult classic and this scene still traumatizing me


In [4]:
!pwd                # shows your current folder
!git status         # check uncommitted changes
!git add .
!git commit -m "Add initial notebook"
!git push origin main


'pwd' is not recognized as an internal or external command,
operable program or batch file.


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


[main 7c99b9d] Add initial notebook
 2 files changed, 261 insertions(+)
 create mode 100644 .ipynb_checkpoints/Untitled-checkpoint.ipynb
 create mode 100644 Untitled.ipynb


To https://github.com/Starlight0901/Reddit_SriLanka_Social_Insight.git
   1d75904..7c99b9d  main -> main
